In [1]:
# Generates a matrix - Drugs along columns and Genes along rows.
def data_preproc():
    import os
    import pandas as pd
    import glob
    import subprocess
    subprocess.run(['/bin/bash',"shell.sh"])
    with open("list_files",'w') as f:
        for ind,fl in enumerate(sorted(glob.glob("proc*.txt"))):
            if ind<1:
                pd_result_matrix=pd.read_csv(fl,'\t',header=None)
                bool_series = pd_result_matrix[0].duplicated(keep = False)     
                pd_result_matrix=(pd_result_matrix[~bool_series])
            else:
                f_pd=pd.read_csv(fl,'\t',header=None)
                bool_series = f_pd[0].duplicated(keep = False) 
                f_pd=f_pd[~bool_series]
                pd_result_matrix=pd.merge(pd_result_matrix,f_pd[[0,1]], how='outer',on=0)
            f.write(fl+"\n")
    
    pd_result_matrix.columns=["Gene.Symbol"]+["drug_"+str(i) for i in range(1,len(glob.glob("proc*.txt"))+1)]        
    pd_result_matrix.fillna(0, inplace = True) 
    pd_result_matrix.to_csv("matrix_drugs")
    with open("final_pilot.txt","w") as f:
        for ind,col in enumerate(pd_result_matrix.columns):
            if ind>0:
                f.write('\n'.join(map(str,list(pd_result_matrix[col]))))
                f.write('\n')
    return(pd_result_matrix)

In [2]:
def data(file_name):
    import numpy as np
    all_data = np.loadtxt(file_name, unpack=True, delimiter='\t')
    data_length = int(all_data.shape[0])
    return(all_data,data_length)

def percent_change(starting_point, current_point):
    """
    Computes the percentage difference between two points
    :return: The percentage change between starting_point and current_point
    """
    default_change = 0.00001
    try:
        change = ((float(current_point) - starting_point) / abs(starting_point)) * 100.00
        if change == 0.0:
            return default_change
        else:
            return change
    except:
        return default_change

def drug_sim_fls():
    flg=0
    counter=1
    import numpy as np
    import pandas as pd
    for out_pos in range(flg,data_length-end_point,end_point):
        print(out_pos,out_pos+end_point)
    #pattern_array=[]
        with open(str(out_pos)+"_"+str(dots_for_pattern)+".txt",'w') as f:
            f.write("Gene_Start_End"+","+",".join('Drug_'+str(counter)+'_'+str(x) for x in range(counter+1,int(len(all_data)/end_point)+1))+"\n")
            for each_drug in range(out_pos,out_pos+end_point):
                pattern=[]
                if each_drug<out_pos+end_point-dots_for_pattern:
                    for index in reversed(range(dots_for_pattern)):
                        point = percent_change(all_data[each_drug], all_data[each_drug + dots_for_pattern - index])
                        pattern.append(point)
                #pattern_array.append(pattern)
                #print("inner:",each_drug)
                #print(int(len(all_data)/end_point))
                    pattern_for_all_rec=[]
            
                    for comp_drug in range(each_drug+end_point,len(all_data),end_point):
                        y=comp_drug
                    #print(y)   
                        pattern_for_rec=[]
                        for index in reversed(range(dots_for_pattern)):
                            point_r = percent_change(all_data[y], all_data[y + dots_for_pattern - index])
                            pattern_for_rec.append(point_r)

                        pattern_for_all_rec.append(pattern_for_rec)
            #print(pattern)
            #print(pattern_for_all_rec)
            #similarity
                
                    similarities_array_all=[]
                    for pat_r in pattern_for_all_rec:
                        similarities_array=[]
                        for index in range(dots_for_pattern):
            # Compute the values of similarity only if it's the first value to be computed, or if the previous one was
            # at least 50% similar
                            similarities_array.append(100.00 - abs(percent_change(pat_r[index], pattern[index])))
                        if (np.sum(np.asarray(similarities_array)>=50)>4):
                            if np.sum(np.asarray(pat_r)!=100)>5:
                                similarities_array_all.append(np.sum(similarities_array) / dots_for_pattern)
                            else:
                                similarities_array_all.append(0)
                        else:
                            similarities_array_all.append(0)
                    f.write(str(each_drug)+"_"+str(abs(each_drug+dots_for_pattern))+","+",".join(str(sim) for sim in similarities_array_all)+"\n")
        counter+=1 

def similar_drugs():
    from scipy import stats
    dic={}
    counter=1
    import pandas as pd
    for i in range(0,data_length-end_point,end_point):
        print(str(i)+"_"+str(dots_for_pattern)+".txt")
        df=pd.read_csv(str(i)+"_"+str(dots_for_pattern)+".txt")
        df=df.rank(axis=1,ascending=False,method='dense')
        ls=[]
        for ind,val in enumerate(df.columns):
            ls.append(stats.kendalltau(np.insert((np.ones(df.shape[0])),0,0),np.insert(list(df[val]),0,0))[0])
        dic["drug_"+str(counter)]=np.concatenate((np.zeros(counter),ls))
        counter+=1
    dic["drug_"+str(counter)]=np.zeros(counter)
    heat=pd.DataFrame(dic)
    tran=heat.transpose()
    heat.index=heat.columns
    tran.columns=tran.index
    htmp=heat.add(tran)
    Each_drg_max=pd.concat([heat.idxmax(),heat.max()],axis=1)
    print(Each_drg_max.nlargest(5,1))

def pattern(d1,d2):
    ele=list(all_data[(d1-1)*end_point:d1*end_point])+list(all_data[(d2-1)*end_point:d2*end_point])
    pattern_array=[]
    recog_pattern=[]
    for i in range(0,end_point-dots_for_pattern):
        print(i,i+end_point)
        pattern=[]
        rec_pattern=[]
        for index in reversed(range(dots_for_pattern)):
            point = percent_change(ele[i], ele[i + dots_for_pattern - index])
            point_r = percent_change(ele[i+end_point], ele[i+end_point + dots_for_pattern - index])
            pattern.append(point)
            rec_pattern.append(point_r)
        pattern_array.append(pattern)
        recog_pattern.append(rec_pattern)

    similarity_array=[]
    index_similarity=[]
    dic={}
    for ln in range(len(pattern_array)):
        similarity=[]
        for index in range(dots_for_pattern):
            similarity.append(100.00 - abs(percent_change(recog_pattern[ln][index], pattern_array[ln][index])))
        similarity_array.append(similarity)
        if (np.sum(np.asarray(similarity)>=50)>5):
            if np.sum(np.asarray(recog_pattern[ln])!=100)>5:
                print(ln,similarity,pattern_array[ln],recog_pattern[ln])
                dic[ln]=[similarity,pattern_array[ln],recog_pattern[ln], np.sum(similarity) / dots_for_pattern]


                
    with open("list_files",'r') as f:
        for d, val_d in enumerate(f):
            if d+1 == int(d1):
                drug1=val_d.split("_")[1]
            elif d+1 == int(d2):
                drug2=val_d.split("_")[1]
                
    genelist=list((pd.read_csv("matrix_drugs"))['Gene.Symbol'])
    import matplotlib.pyplot as plt
    for key,val in dic.items():
        xp = np.arange(0, dots_for_pattern, 1)
        plt.figure(figsize=(10, 4))
        plt.plot(genelist[key:key+dots_for_pattern], val[1], 'red', linewidth=5)
        plt.plot(genelist[key:key+dots_for_pattern], val[2], '#24BC00', linewidth=5)
        plt.xlabel("Genes",fontsize=16)
        plt.ylabel("Score",fontsize=16)
        plt.legend([drug1,drug2], loc='upper right',prop={'size': 8})
        plt.savefig(str(val[3])+".png", dpi = 72)
        print(val[3])

    kegg=[]
    for i in dic.keys():
        kegg=kegg+genelist[i:i+10]

    with open("go.txt",'w') as f:
        f.write("\n".join(item for item in kegg))


In [4]:
if __name__=="__main__":
    import numpy as np
    import pandas as pd
    dots_for_pattern=int(input("Enter the window length for genes\n"))
    end_point=data_preproc().shape[0]
    all_data,data_length=data("final_pilot.txt")
    drug_sim_fls()
    similar_drugs()
    drug1=int(input("Enter drug - No 1\n"))
    drug2= int(input("Enter drug - No 2 \n"))
    pattern(drug1,drug2)

Enter the window length for genes
10
0 20307


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in greater_equal


0_10.txt
             0    1
drug_1  drug_2  1.0
drug_2  drug_1  0.0
Enter drug - No 1
1
Enter drug - No 2 
2
0 20307
1 20308
2 20309
3 20310
4 20311
5 20312
6 20313
7 20314
8 20315
9 20316
10 20317
11 20318
12 20319
13 20320
14 20321
15 20322
16 20323
17 20324
18 20325
19 20326
20 20327
21 20328
22 20329
23 20330
24 20331
25 20332
26 20333
27 20334
28 20335
29 20336
30 20337
31 20338
32 20339
33 20340
34 20341
35 20342
36 20343
37 20344
38 20345
39 20346
40 20347
41 20348
42 20349
43 20350
44 20351
45 20352
46 20353
47 20354
48 20355
49 20356
50 20357
51 20358
52 20359
53 20360
54 20361
55 20362
56 20363
57 20364
58 20365
59 20366
60 20367
61 20368
62 20369
63 20370
64 20371
65 20372
66 20373
67 20374
68 20375
69 20376
70 20377
71 20378
72 20379
73 20380
74 20381
75 20382
76 20383
77 20384
78 20385
79 20386
80 20387
81 20388
82 20389
83 20390
84 20391
85 20392
86 20393
87 20394
88 20395
89 20396
90 20397
91 20398
92 20399
93 20400
94 20401
95 20402
96 20403
97 20404
98 20405
99 20406


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


21391
1085 21392
1086 21393
1087 21394
1088 21395
1089 21396
1090 21397
1091 21398
1092 21399
1093 21400
1094 21401
1095 21402
1096 21403
1097 21404
1098 21405
1099 21406
1100 21407
1101 21408
1102 21409
1103 21410
1104 21411
1105 21412
1106 21413
1107 21414
1108 21415
1109 21416
1110 21417
1111 21418
1112 21419
1113 21420
1114 21421
1115 21422
1116 21423
1117 21424
1118 21425
1119 21426
1120 21427
1121 21428
1122 21429
1123 21430
1124 21431
1125 21432
1126 21433
1127 21434
1128 21435
1129 21436
1130 21437
1131 21438
1132 21439
1133 21440
1134 21441
1135 21442
1136 21443
1137 21444
1138 21445
1139 21446
1140 21447
1141 21448
1142 21449
1143 21450
1144 21451
1145 21452
1146 21453
1147 21454
1148 21455
1149 21456
1150 21457
1151 21458
1152 21459
1153 21460
1154 21461
1155 21462
1156 21463
1157 21464
1158 21465
1159 21466
1160 21467
1161 21468
1162 21469
1163 21470
1164 21471
1165 21472
1166 21473
1167 21474
1168 21475
1169 21476
1170 21477
1171 21478
1172 21479
1173 21480
1174 21481
1175

...

1702 [-14.10662945366687, 54.60263520163607, 73.97775567046253, 53.39901707066032, 99.30836063829622, 79.57114214669225, -46.71916750791016, -60.27793611327564, 68.2810311726782, 60.93556944302503] [18.456410234020417, -30.327970135354064, -127.91943414792762, -16.533990078891105, -156.73493726834567, -121.44480141808141, -153.91252046049394, 225.62720046249254, -100.1663278154537, -102.61365202742108] [-130.83501125935405, -55.54305213175012, -172.91607860848183, -30.963098172785614, -155.6583428990798, -100.84360474962834, -62.38368993181662, -374.3114230694441, -146.69715160296818, -168.39696906970764]
2523 [81.21510179880401, 85.9915993778151, 98.09800938133701, -49.28088856406981, 57.446139376651324, 73.82981246097137, 35.32407545812838, 14.986507163233554, 66.2401280177011, 44.64766726023303] [90.71653899965648, 90.32856257109256, 66.56258903860058, -78.53141298930163, 143.3680841313452, 102.22458435709912, 38.03649749200104, 27.623615102081363, 117.47551723920091, 79.74532158675

10607 [3.661606102658723, 22.39095573651059, 98.39522910219475, 74.95194574037306, 99.12222213536492, 74.02289949414998, 25.474918672828494, -763.7770953981667, 91.83982781892276, 69.49270625281736] [124.14854198361243, 26.445184753557914, 100.07452344981118, 150.7074976292238, 161.24764130836965, 68.54599020225643, 188.21928010138745, 305.28160712700594, 98.51859444390246, 78.95834486862078] [63.23192296689842, 118.10654741475157, 98.4939216589217, 120.51966623672712, 162.67557146586563, 92.60106084830359, 107.84655057604257, 31.67554080551006, 107.27218983701492, 60.501097372824276]
12083 [35.33099123617286, -5.488315770546535, 93.00333614353531, 67.9229276190643, -18.445312035883887, 69.36070173767972, 78.28038399733981, 94.88487686118542, 80.72288885115816, 82.91669419361294] [-133.48486454101425, -194.85576625947124, -90.74047035630038, -77.33707879628095, -220.6928407396045, -66.61819341521092, -97.47579501719548, -111.19415414599881, -106.90842071943669, -81.62841527102839] [-81

17888 [91.55482278762547, 50.553787519632074, 44.60395853317374, -50.74772689791257, 75.8236172612704, 16.454800200998903, 61.403912185421916, 76.48775644511032, 80.29517908226961, -104.03602180451963] [-123.32118679748037, -149.47031825720268, -99.54092381636787, -144.11621266234232, -111.84069880267083, -50.80267555276784, -96.13900950083553, -118.92194725692273, -117.02419918096534, -90.2607593094197] [-134.6965490649701, -100.01613006876164, -223.1661204292783, -57.47458389563652, -90.06599832915622, -308.7407621618148, -156.56821541032068, -96.28352933616092, -145.74249726881305, 86.759141443352]
18283 [67.51309839381872, -5665.352435199944, 91.17141824820531, 98.02476028176267, 82.81969210306492, -6.661125131683704, 44.9727236158901, 63.98051386797724, 51.6568563420764, 17.009561692012284] [80.52509003422666, 180.63873213276668, 69.45746156843636, 169.96393666951545, 167.6334904433691, -12.52356995257467, 130.33796721069152, 109.71694546320987, 153.39259776493947, 11.421428021844

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


57.84891728333181
48.41400535188018
-628.8284701091449
55.991077034801776
56.32467264577093
39.10133484110848
-220.9835956070885
-50.52456783350997
-291.1737380440555
-155.60572484261561
42.478254269320075
-31.82488270243556
55.795434634246384
-143.53074733851753
43.73266473618943
37.418156891953636
49.737289068982975
34.23940853130703
-515.486493578682
-4.586663654846623
9.836499185685028
67.17606075579964
